In [ ]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [ ]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

# Read in Mutual Fund informaiton csv


In [ ]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv")
,index_col="fund_symbol")
mutualFunds.head()

In [ ]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
#convert dates to date
mutualFunds["inception_date"] = pd.to_datetime(mutualFunds["inception_date"])
mutualFunds["management_start_date"] = pd.to_datetime(mutualFunds["management_start_date"])
mutualFunds["returns_as_of_date"] = pd.to_datetime(mutualFunds["returns_as_of_date"])

In [ ]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

In [ ]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

_We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows._

In [ ]:
mutualFunds.dropna(axis = 0)

If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [ ]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [ ]:
# Handling Missing Values
# Fill missing values in numerical columns with mean or median
numerical_cols = mutualFunds.select_dtypes(include=['number']).columns
mutualFunds[numerical_cols] = mutualFunds[numerical_cols].fillna(mutualFunds[numerical_cols].mean())

# Fill missing values in categorical columns with mode
categorical_cols = mutualFunds.select_dtypes(include=['object']).columns
mutualFunds[categorical_cols] = mutualFunds[categorical_cols].fillna(mutualFunds[categorical_cols].mode().iloc[0])

In [ ]:
# Encoding Categorical Variables
# One-hot encode categorical variables
mutualFunds_encoded = pd.get_dummies(mutualFunds, columns=categorical_cols)

# Scaling Numerical Features
# Separate numerical columns for scaling
numerical_cols = mutualFunds_encoded.select_dtypes(include=['number']).columns

In [ ]:
list[0]

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [ ]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in range(len(list)):
    if list[i][1][0] >= threshold:
        drop_columns.append(list[i][0])

In [ ]:
print(len(drop_columns))
drop_columns


In [ ]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

In [ ]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
#Drop unnecessary columns
mutualFunds.drop(columns=["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy"],inplace=True)


In [ ]:
#Remove quarterly data
mutualFunds.drop(columns=["fund_return_2021_q1",
"fund_return_2020_q4","fund_return_2020_q3","fund_return_2020_q2","fund_return_2020_q1",
"fund_return_2019_q4","fund_return_2019_q3","fund_return_2019_q2","fund_return_2019_q1",
"fund_return_2018_q4","fund_return_2018_q3","fund_return_2018_q2","fund_return_2018_q1",
"fund_return_2017_q4","fund_return_2017_q3","fund_return_2017_q2","fund_return_2017_q1",
"fund_return_2016_q4","fund_return_2016_q3","fund_return_2016_q2","fund_return_2016_q1",
"fund_return_2015_q4","fund_return_2015_q3","fund_return_2015_q2","fund_return_2015_q1",
"fund_return_2014_q4","fund_return_2014_q3","fund_return_2014_q2","fund_return_2014_q1",
"fund_return_2013_q4","fund_return_2013_q3","fund_return_2013_q2","fund_return_2013_q1",
"fund_return_2012_q4","fund_return_2012_q3","fund_return_2012_q2","fund_return_2012_q1",
"fund_return_2011_q4","fund_return_2011_q3","fund_return_2011_q2","fund_return_2011_q1",
"fund_return_2010_q4","fund_return_2010_q3","fund_return_2010_q2","fund_return_2010_q1",
"fund_return_2009_q4","fund_return_2009_q3","fund_return_2009_q2","fund_return_2009_q1",
"fund_return_2008_q3","fund_return_2008_q2","fund_return_2008_q1",
"fund_alpha_3years","fund_beta_3years","fund_mean_annual_return_3years","fund_r_squared_3years","fund_stdev_3years","fund_sharpe_ratio_3years","fund_treynor_ratio_3years",
"fund_alpha_5years","fund_beta_5years","fund_mean_annual_return_5years","fund_r_squared_5years","fund_stdev_5years","fund_sharpe_ratio_5years","fund_treynor_ratio_5years",
"fund_alpha_10years","fund_beta_10years","fund_mean_annual_return_10years","fund_r_squared_10years","fund_stdev_10years","fund_sharpe_ratio_10years","fund_treynor_ratio_10years",
"fund_return_category_rank_ytd","fund_return_category_rank_1month","fund_return_category_rank_3months","fund_return_category_rank_1year","fund_return_category_rank_3years",
"fund_return_category_rank_5years","load_adj_return_1year","load_adj_return_3years","load_adj_return_5years","load_adj_return_10years",
"top10_holdings"],inplace=True)

In [ ]:
#Remove metrics that we can caluclate ourselves
#keep: "esg_score","environment_score", "sustainability_score", "sustainability_rank",   "social_score",  "governance_score", 
mutualFunds.drop(columns=["esg_peer_count","peer_esg_min", "peer_esg_avg", "peer_esg_max",
"peer_environment_min", "peer_environment_avg", "peer_environment_max", 
"peer_social_min", "peer_social_avg", "peer_social_max",
"peer_governance_min", "peer_governance_avg", "peer_governance_max"],inplace=True)


In [ ]:
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

Done with Data cleanup and preprocessing


In [ ]:
# Calculate the average for each column
column_means = mutualFunds.mean(skipna=True, numeric_only=True)
# Replace NaN values in each column with the respective column average
mutualFunds.fillna(column_means, inplace=True)

In [ ]:
mutualFunds.dropna(inplace= True)
mutualFunds

In [ ]:
mutualFunds.dtypes

# Binning!

In [ ]:
mutualFunds.loc[:,mutualFunds.dtypes == "object"].dtypes

Binning `fund_category`

In [ ]:
fund_category_type_count = mutualFunds["fund_category"].value_counts()[mutualFunds["fund_category"].value_counts() > 150]
print(100*fund_category_type_count.sum()/len(mutualFunds))
print(fund_category_type_count.count())

In [ ]:
for cat in mutualFunds["fund_category"]:
    if cat in fund_category_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_category"] = mutualFunds["fund_category"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_category"].value_counts()

Binning `fund_family_type_count`

In [ ]:
fund_family_type_count = mutualFunds["fund_family"].value_counts()[mutualFunds["fund_family"].value_counts() > 150]
print(100*fund_family_type_count.sum()/len(mutualFunds))
print(fund_family_type_count.count())

In [ ]:
for cat in mutualFunds["fund_family"]:
    if cat in fund_family_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_family"] = mutualFunds["fund_family"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_family"].value_counts()

Binning `esg_peer_group`

In [ ]:
#finding threshhold to bin esg_peer_group
esg_peer_type_count = mutualFunds["esg_peer_group"].value_counts()[mutualFunds["esg_peer_group"].value_counts() > 100]
print(100* esg_peer_type_count.sum()/len(mutualFunds))
print(esg_peer_type_count.count())

In [ ]:
#check the bins of esg_peer_group
esg_peer_type_count.index.to_list()
#replace bins in dataframe for esg_peer_group
for esg in mutualFunds["esg_peer_group"]:
    if esg in esg_peer_type_count.index.to_list():
        next
    else:
        mutualFunds["esg_peer_group"] = mutualFunds["esg_peer_group"].replace(esg, "Other")

In [ ]:
mutualFunds["esg_peer_group"].value_counts()

In [ ]:
mutualFunds

# SQLite Database 

In [ ]:
import os
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score 

In [ ]:
# Define the relative path for the SQLite database
db_relative_path = "mutual_funds_data.db"

In [ ]:
# Connect to SQLite database with relative path
db_path = os.path.join(os.getcwd(), db_relative_path)
with sqlite3.connect(db_path) as conn:
    # Query data from SQLite database
    query = "SELECT * FROM mutual_funds_data"
    mutualFunds = pd.read_sql_query(query, conn)


# Data Implementation - 5 Features

In [ ]:
# Define feature columns
feature_columns = [
    'fund_yield',
    'total_net_assets',
    'morningstar_overall_rating',
    'fund_annual_report_net_expense_ratio',
    'fund_sector_technology'
]

In [ ]:
# Create X and y
X = mutualFunds[feature_columns]
y = mutualFunds['year_to_date_return']

In [ ]:
# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [ ]:
#  Training
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

### Model Testing

In [ ]:
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

### Analysis:

For the first data implementation, we've selected the following five feature columns:
1. 'fund_yield'
2. 'total_net_assets'
3. 'morningstar_overall_rating'
4. 'fund_annual_report_net_expense_ratio'
5. 'fund_sector_technology'
We used these features to predict the 'year_to_date_return' of mutual funds. After splitting the data into training and testing sets, we trained a RandomForestRegressor model with a random state of 42.

The model achieved a Mean Squared Error (MSE) of 0.00240 and a R-squared (R2): 0.634. This means that the model explains approximately 63.4% of the variance in the 'year_to_date_return' target variable, which is a decent performance. With an accuracy of only 63.4%, we decided to explore additional features or transformations of existing features that could better capture the relationship with the target variable.

# Data Implementation - Grid search with Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of trees in the forest
    'max_depth': [None, 10, 20],       # Maximum depth of the trees
    'min_samples_split': [2, 5, 10]    # Minimum number of samples required to split a node
}

# Initialize the RandomForestRegressor model
model = RandomForestRegressor(random_state=42)


### Model Training

In [ ]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

### Model Testing

In [ ]:
# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

### Analysis:

After conducting a grid search with cross-validation, we found the best hyperparameters for our Random Forest Regressor model. The optimal parameters were determined to be:

max_depth: None
min_samples_split: 2
n_estimators: 150
These parameters suggest that allowing the trees to grow without a maximum depth (max_depth set to None), while requiring a minimum of 2 samples to split a node (min_samples_split), and using an ensemble of 150 trees (n_estimators) yielded the best performance.

Using these parameters, we evaluated the model on the test set and obtained the following results:
Mean Squared Error (MSE): 0.00242
R-squared (R2): 0.631
These metrics provide insights into the model's performance. The MSE indicates the average squared difference between the predicted and actual values, with lower values indicating better performance. Meanwhile, the R-squared value represents the proportion of variance in the target variable that is predictable from the independent variables. An R-squared value closer to 1 suggests that the model explains a large portion of the variance in the target variable. Overall, the model seems to perform reasonably well, with a relatively low MSE and a moderate R-squared value. However, there is still room for improvement, and further analysis and fine-tuning may be necessary to enhance the model's performance further.

# Data Implementation - Float Columns

In [ ]:
# Filter float columns
float_columns = mutualFunds.select_dtypes(include=['float']).columns.tolist()

# Create X and y
X = mutualFunds[float_columns]
y = mutualFunds['year_to_date_return']

In [ ]:
# Preprocess data
X.fillna(X.mean(), inplace=True)

In [ ]:
# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model Training

In [ ]:
#  Training
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

### Model Testing

In [ ]:
# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Analysis 

The data implementation initially filters out float columns from the mutual funds dataset, and missing values within these columns are imputed with their respective means. While the warning message regarding setting values on a DataFrame slice during preprocessing is noted, it doesn't seem to affect subsequent operations. Following this, the model is trained using a Random Forest Regressor with default parameters. During testing, the model showcases exceptional performance, achieving a remarkably low Mean Squared Error of approximately 2.03e-07 and an impressively high R-squared value of around 0.99997. These metrics indicate an almost perfect fit of the model to the data, suggesting strong predictive capabilities. However, further investigation into the warning message and potentially implementing safer practices for data preprocessing would enhance the robustness of the analysis.

# Predictions

In [ ]:
# Create a new column in your test dataset for the predictions
X_test['predicted_year_to_date_return'] = predictions

# Check the updated test dataset
print(X_test.head(5))
